<a href="https://colab.research.google.com/github/Zezake/Web_Scraping/blob/master/WebScrapping_ElPais_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codi del Dataset


> Aleix Martinez i Ignasi Vilarasau






In [0]:
"""Importem les llibreries necessaries per executar el codi"""

#Webscrapping
import requests
from bs4 import BeautifulSoup
#Per incloure una columna amb el moment que s'ha fet l'extracció
from datetime import datetime
#Extracció de fitxer del Colab
from google.colab import files

In [0]:
"""  Neteja del text
Extreiem els salts de pàgina que poden apareixer dins dels diferents camps.
A més a més extreiem els signes de puntuació que poden interferir amb el format del fitxer csv
"""
def neteja_text(text_html):
  try:
    text_html = text_html.text.strip().replace('\n','')
    text_html = treure_puntuacio(text_html)
  except:
    text_html = 'NA'
  
  return text_html

def treure_puntuacio(text):
  puntuacio = [',',';']
  for p in puntuacio:
    text = text.replace(p,'')
  return text

In [0]:
"""
Funcions per extreure el tipus i subtipus de cada secció
"""
def get_section_subtype(section):
  section_name = section.find('a',{'class':'enlace'})
  return neteja_text(section_name)
  

def get_section_type(section):
  section_type = section.find('h3')
  return neteja_text(section_type)

In [0]:
"""
Funcions per extreure la informació bàsica de cada article.
Aquestes són l'autor, el titol, la localització i el número de comentaris.
"""

def get_article_author(article):
  article_author = article.find('span',{'itemprop':'name'})
  #article_author = article.find('div', {'class':'firma'})
  return neteja_text(article_author)

def get_article_title(article):
  article_title = article.find('h2')
  return neteja_text(article_title)

def get_article_location(article):
  article_location = article.find('span',{'class':'articulo-localizacion'})
  return neteja_text(article_location)

def get_article_comments(article):
  article_comments = article.find('span',{'class':'articulo-comentarios'})
  return neteja_text(article_comments)

In [0]:
"""
Funcions per extreure la informacio de les fotos dels articles.
Consisteix en extreure el peu de foto i l'autor de la imatge.
"""

def get_article_foto_text(article):
  article_foto_text = article.find('span',{'class':'foto-texto'})
  return neteja_text(article_foto_text)

def get_article_foto_author(article):
  article_foto_author = article.find('span',{'class':'foto-firma'})
  return neteja_text(article_foto_author)

In [0]:
"""
Funció especifica per extreure la data de creació de la noticia.
La data es retornada en el format seguent: YYYY-MM-DDThh:mm:ss
D'ella extreiem el dia-mes-any i l'hora de creació de l'article.
"""
def get_article_date(article):
  article_data = article.findAll('meta')
  date = [_.get('content', None) for _ in article_data][-2]
  hour, new_date = extract_date(date)
  return hour, new_date

def extract_date(date):
  year = date[:4]
  month = date[5:7]
  day = date[8:10]
  new_date = '-'.join([day,month,year])
  hour = date[11:13]
  return hour, new_date

In [0]:
"""
Funció que serveix per cridar les funcions anteriorment descrites.
L'objectiu és crear una llista que agrupi tota la informació 
"""

def data_from_article(article):
  a_title = get_article_title(a)
  a_author = get_article_author(a)
  a_location = get_article_location(a)
  a_comments = get_article_comments(a)
  a_hour, a_date = get_article_date(a)
  a_foto_text = get_article_foto_text(a)
  a_foto_author = get_article_foto_author(a)
  a_extraction_date = datetime.today().strftime('%d-%m-%Y')
  a_extraction_hour = datetime.today().strftime('%H')
  Data = [a_extraction_date, a_extraction_hour, a_date, a_hour,
         a_title, a_author, a_location,
         a_comments, a_foto_author, a_foto_text]
  
  
  return Data

In [0]:
#Extreiem l'hora en la qual es farà el webscrapping
today = datetime.today().strftime('%d-%m-%Y')
hour = datetime.today().strftime('%H')

#Generem un nom del fitxer amb la seguent regla: (nom del dataset)_(hora extracció)_(data_extracció).txt
filename = 'dataset_elpais_'+hour+'_'+today+'.txt'

#Extreiem el HTML i fem el parsing amb bs4
url = "https://www.elpais.com"
html = requests.get(url)
codi_html = html.content
soup = BeautifulSoup(codi_html, 'html.parser')

#Busquem les diferents seccions del diari
sections = soup.findAll('section', limit = None)


In [0]:
#Obrim el fitxer
with open(filename,'w') as f:
  #Escribim el header on indiquem cada columna del dataset separada per comes, fitxer CSV.
  f.write("""Extraction Date,Extraction Hour,Publication Date,Publication Hour,Title,Author,Location,Num Comments,Photo Author,Photo Text,Sectiom,Subsection\n""")

  #Recorrem iterativament sobre totes les seccions del diari
  for i, s in enumerate(sections):
    #Extreiem informació respecte la secció
    s_type = get_section_type(s)
    s_subtype = get_section_subtype(s)
    
    #Informem de quina secció es procedeix a llegir
    print('{} \t  Secció: {}'.format(i, s_type))
    
    #Busquem els articles de cada secció i iterem sobre cadascún d'ells
    articles = s.findAll('article')
    for j, a in enumerate(articles):
      
      #Etreiem tota la informació de l'article en una llista
      data = data_from_article(a)
      #Afegim la secció a la qual pertany l'article
      data.append(s_type)
      data.append(s_subtype)
      #Convertim els elements de la llista en un array separant cada columna per comes
      data_to_write = ','.join(data)
      
      #Escribim la informació al fitxer
      f.write(data_to_write+'\n')
      
#Descarregem el fitxer.
files.download(filename)

0 	  Secció: NA
1 	  Secció: Destacados
2 	  Secció: opinión
3 	  Secció: AMÉRICAS
4 	  Secció: INTERNACIONAL
5 	  Secció: ¿Y tú qué piensas?
6 	  Secció: El País Economía
7 	  Secció: Sociedad
8 	  Secció: Cultura
9 	  Secció: Deportes
10 	  Secció: ESPAÑA
11 	  Secció: Verne
12 	  Secció: Ciencia y Tecnología
13 	  Secció: Estilo y Vida
14 	  Secció: UN PAÍS DE BLOGS
15 	  Secció: VÍDEOS Y FOTOS
16 	  Secció: DESDE EL PAÍS
17 	  Secció: ONE
18 	  Secció: Entrevista W
19 	  Secció: TERMÓMETRO ECONÓMICO Y SOCIAL DE AMÉRICA
20 	  Secció: EL PAÍS IN ENGLISH
21 	  Secció: Lo más visto
22 	  Secció: NA
